## traditional

In [1]:
import numpy as np

def make_cluster(natoms, radius=20, seed=1981):
    np.random.seed(seed)
    arr = np.random.normal(0, radius, size=(natoms,3))-0.5
    return arr

In [2]:
class lj_pure(object):
    
    @classmethod
    def lj(cls, r):
        sr6 = (1./r)**6
        pot = 4.*(sr6*sr6 - sr6)
        return pot


    @classmethod
    def distance(cls, atom1, atom2):
        dx = atom2[0] - atom1[0]
        dy = atom2[1] - atom1[1]
        dz = atom2[2] - atom1[2]

        r = (dx*dx + dy*dy + dz*dz)**0.5
        return r


    @classmethod
    def potential(cls, cluster):
        energy = 0.0
        for i in range(len(cluster)-1):
            for j in range(i+1,len(cluster)):
                r = cls.distance(cluster[i],cluster[j])
                e = cls.lj(r)
                energy += e
        return energy

In [3]:
import numpy as np
class lj_numpy(object):
    
    @classmethod
    def lj(cls, r):
        sr6 = (1./r)**6
        pot = 4.*(sr6*sr6 - sr6)
        return pot
    
    
    @classmethod
    def distances(cls, cluster):
        diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
        mat = np.sqrt((diff*diff).sum(-1))
        return mat

    
    @classmethod
    def potential(cls, cluster):
        d = cls.distances(cluster)
        pot = cls.lj(d)
        energy = np.nansum(pot) / 2
        return energy

In [4]:
cluster = make_cluster(int(2e3), radius=100)

In [5]:
lj_pure.potential(cluster)

-1.3431786584748961

In [6]:
lj_numpy.potential(cluster)

/Users/albert/Applications/miniconda3/envs/cuda/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in true_divide
/Users/albert/Applications/miniconda3/envs/cuda/lib/python3.6/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in subtract


-1.3431786584750052

## numba

In [7]:
import numba

@numba.vectorize(['float64(float64)'], nopython=True, target='parallel')
def ulj(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot

@numba.jit(nopython=True)
def dist(atom1, atom2):
    dx = atom2[0] - atom1[0]
    dy = atom2[1] - atom1[1]
    dz = atom2[2] - atom1[2]

    r = np.sqrt(dx*dx + dy*dy + dz*dz)
    return r


@numba.guvectorize(['(float64[:,:], float64[:,:])'], '(n,m)->(n,n)', nopython=True, target='parallel')
def distance_matrix(cluster, dmat):
    for i in range(len(cluster)-1):
        dmat[i,i] = 0.0
        for j in range(i+1,len(cluster)):
            r = dist(cluster[i],cluster[j])
            dmat[i,j] = r
            dmat[j,i] = r

def upotential(cluster):
    n = cluster.shape[0]
    dmat = np.empty(shape=(n,n), dtype=cluster.dtype)
    distance_matrix(cluster, dmat)
    
    pot = ulj(dmat)
    energy = np.nansum(pot) / 2.
    
    return energy

In [8]:
upotential(cluster)

-1.3431786584750047

## cuda

In [9]:
import numba
from numba import cuda
import math

@numba.vectorize(['float64(float64)'], target='cuda')
def vlj(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot

@cuda.jit(device=True)
def glj(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot

@cuda.jit(device=True)
def gdist(atom1, atom2):
    dx = atom2[0] - atom1[0]
    dy = atom2[1] - atom1[1]
    dz = atom2[2] - atom1[2]

    r = math.sqrt(dx*dx + dy*dy + dz*dz)
    return r


@numba.guvectorize(['void(float64[:,:], float64[:,:])'], '(n,m)->(n,n)', target='cuda')
def gdistance_matrix(cluster, dmat):
    for i in range(len(cluster)-1):
        dmat[i,i] = 0.0
        for j in range(i+1,len(cluster)):
            dmat[j,j] = 0.0
            r = gdist(cluster[i],cluster[j])
            p = glj(r)
            dmat[i,j] = p
            dmat[j,i] = p
            
@cuda.reduce
def my_sum(a,b):
    return a + b

In [10]:
#%%time
#g_cluster = cuda.to_device(cluster)
#
#gdistance_matrix.max_blocksize = 32
#pmat = gdistance_matrix(g_cluster)
#
#my_sum(pmat.ravel()) / 2

In [11]:
#big_cluster = make_cluster(int(1e2), radius=10)
big_cluster = make_cluster(int(5e3), radius=100)

In [12]:
%timeit lj_numpy.potential(big_cluster)
p = lj_numpy.potential(big_cluster)
print(p)

/Users/albert/Applications/miniconda3/envs/cuda/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in true_divide
/Users/albert/Applications/miniconda3/envs/cuda/lib/python3.6/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in subtract


2.65 s ± 63.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
-2.99938117826


In [13]:
%%time
g_cluster = cuda.to_device(big_cluster)

gdistance_matrix.max_blocksize = 32
pmat = gdistance_matrix(g_cluster)

p = my_sum(pmat.ravel()) / 2

CudaAPIError: [702] cuModuleLoadDataEx error:


In [ ]:
print(p)

## thanks, Intel!

https://www.anaconda.com/blog/developer-blog/parallel-python-with-numba-and-parallelaccelerator/

In [ ]:
import numba

@numba.jit(nopython=True)
def lj(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot


@numba.jit(nopython=True)
def distance(atom1, atom2):
    dx = atom2[0] - atom1[0]
    dy = atom2[1] - atom1[1]
    dz = atom2[2] - atom1[2]

    r = np.sqrt(dx*dx + dy*dy + dz*dz)
    return r


@numba.jit(nopython=True, parallel=True)
def potential(cluster):
    energy = 0.0
    for i in numba.prange(len(cluster)-1):
        for j in range(i+1,len(cluster)):
            r = distance(cluster[i],cluster[j])
            e = lj(r)
            energy += e
    return energy

In [ ]:
z = np.zeros((1,3))
%time potential(z)

In [ ]:
%timeit potential(cluster)

## bigger data structures

In [ ]:
import xarray as xr

In [ ]:
a = list('HCNO')
labels = np.random.choice(list('HCNO'), size=cluster.shape[0])

traj = np.stack([cluster, cluster+0.001, cluster-0.02])

In [ ]:
positions = xr.DataArray(traj,
                        dims=('time','atom','position'),
                        coords={'atom':labels,
                                'position':['x','y','z'],
                                'time':range(traj.shape[0])})
positions

In [ ]:
positions.sel(atom='C')

In [ ]:
%timeit potential(positions.sel(time=0).values)